# **Tabulation**

## **Load Dataset & Depedencies**


In [52]:
import re
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import seaborn as sns

## **Coding Kategori Level Risiko**

Diambil dari kolom `Risk_Level` dengan value berisi integer 1-5, kemudian dikategorikan berdasarkan pelabelan `BCM Institute`


In [62]:
data = pd.read_csv("csv_result-Final Dataset.csv")
df = pd.DataFrame(data)
display(df.head(7))


def categorized_risk(df):
    bins = [1, 2, 3, 4, 5, np.inf]
    labels = [
        "(1) Very Low Risk",
        "(2) Low Risk",
        "(3) Medium Risk",
        "(4) High Risk",
        "(5) Very High Risk",
    ]
    df["Risk_Level_Category"] = pd.cut(
        df["Risk_Level"], bins=bins, labels=labels, right=False
    )
    return df


def categorized_duration(df):
    bins = [1, 6, 11, 16, 20, np.inf]
    labels = [
        "1-5 days",
        "6-10 days",
        "11-15 days",
        "16-20 days",
        ">20 days",
    ]
    df["Fixing_Duration_Category"] = pd.cut(
        df["Fixing_Duration"], bins=bins, labels=labels, right=False
    )
    return df


df = categorized_risk(df)
df = categorized_duration(df)

,id,Requirements,Project,Requirement,Risk,Probability,Magnitude,Impact,Dimension,Afftecting,Fixing_Duration,Fix_Cost,Priority,Risk_Level
0,1,The system shall display all the products that...,Transaction Processing System,Functional,Budget,10,Negligible,high,Requirements,9,1,10,95.715385,5
1,2,The system shall allow user to select the prod...,Transaction Processing System,Functional,Quality,22,Very Low,catastrophic,Requirements,7,2,11,35.900000,2
2,3,The system shall display all the available com...,Transaction Processing System,Functional,Schedule,33,Low,high,User,5,1,3,35.692308,2
3,4,The system shall enable user to add one or mor...,Transaction Processing System,Functional,Personal,40,Medium,moderate,Project complexity,8,3,2,44.100000,3
4,5,The system shall notify the user about any con...,Transaction Processing System,Functional,Budget,100,High,Low,planning and control,4,2,1,85.776923,5
5,6,The system shall allow user to update the conf...,Transaction Processing System,Functional,Performance,55,Very High,insignificant,Team,6,4,4,56.453846,3
6,7,The system shall allow user to confirm the com...,Transaction Processing System,Functional,FunctionalValidity,44,Extreme,moderate,Organizational Environment,2,1,2,41.530769,3


In [54]:
Risk_Level = pd.crosstab(
    index=df[df["Project"] == "Transaction Processing System"]["Risk_Level_Category"],
    columns="count",
)
Risk_Level["Percentage"] = (
    Risk_Level["count"] / Risk_Level["count"].sum() * 100
).round(2)
total_row = pd.DataFrame(
    {
        "count": [Risk_Level["count"].sum()],
        "Percentage": [Risk_Level["Percentage"].sum().round(1)],
    },
    index=["Total"],
)
Risk_Level = pd.concat([Risk_Level, total_row])
Risk_Level = Risk_Level.reset_index()
Risk_Level = Risk_Level.rename(columns={"index": "Risk Level"})
Risk_Level = Risk_Level[["Risk Level", "count", "Percentage"]]
display(Risk_Level)

Duration_Level = pd.crosstab(
    index=df[df["Project"] == "Transaction Processing System"][
        "Fixing_Duration_Category"
    ],
    columns="count",
)
Duration_Level["Percentage"] = (
    Duration_Level["count"] / Duration_Level["count"].sum() * 100
).round(2)
total_row = pd.DataFrame(
    {
        "count": [Duration_Level["count"].sum()],
        "Percentage": [Duration_Level["Percentage"].sum().round(1)],
    },
    index=["Total"],
)
Duration_Level = pd.concat([Duration_Level, total_row])
Duration_Level = Duration_Level.reset_index()
Duration_Level = Duration_Level.rename(columns={"index": "Duration Level"})
Duration_Level = Duration_Level[["Duration Level", "count", "Percentage"]]
display(Duration_Level)

,Risk Level,count,Percentage
0,(1) Very Low Risk,5,4.24
1,(2) Low Risk,40,33.90
2,(3) Medium Risk,41,34.75
3,(4) High Risk,23,19.49
4,(5) Very High Risk,9,7.63
5,Total,118,100.00


,Duration Level,count,Percentage
0,1-5 days,80,67.80
1,6-10 days,37,31.36
2,11-15 days,1,0.85
3,Total,118,100.00


## **Tabel Tabulasi Level Risiko**

Create tables to see the relationship between two variables, such as `Magnitude` with `Project Category` and `Requirement Type`. This can reveal, for example, if one project category consistently has a higher proportion of "high-risk" items.


In [ ]:
# Filter the rows first (if needed)
filtered_df = df[
    (df["Risk"] == "Transaction Processing System")
    | (df["Fix_Cost"] == "Transaction Processing System")
]

# Crosstab between Risk and Fix_Cost
Risk_Project = pd.crosstab(index=filtered_df["Risk"], columns=filtered_df["Fix_Cost"])
Risk_Project["Total"] = Risk_Project.sum(axis=1).astype(int)
Risk_Project["Percentage"] = Risk_Project["Total"] / Risk_Project["Total"].sum() * 100
total_row = Risk_Project.sum(axis=0).to_frame().T
total_row.index = ["Total"]
Risk_Project = pd.concat([Risk_Project, total_row])
Risk_Project = Risk_Project.reset_index()
Risk_Project = Risk_Project.rename(columns={"index": "Risk Type"})
# Risk_Project = Risk_Project.drop(columns=["Risk_Level"])
display(Risk_Project)

ValueError: 2

In [59]:
Risk_Project = pd.crosstab(index=df["Risk"], columns=df["Fixing_Duration_Category"])
Risk_Project["Total"] = Risk_Project.sum(axis=1).astype(int)
Risk_Project["Percentage"] = Risk_Project["Total"] / Risk_Project["Total"].sum() * 100
total_row = Risk_Project.sum(axis=0).to_frame().T
total_row.index = ["Total"]
Risk_Project = pd.concat([Risk_Project, total_row])
Risk_Project = Risk_Project.reset_index()
Risk_Project = Risk_Project.rename(columns={"index": "Requirement Type"})
display(Risk_Project)

Fixing_Duration_Category,Requirement Type,1-5 days,6-10 days,11-15 days,>20 days,Total,Percentage
0,Budget,6.0,5.0,0.0,0.0,11.0,3.678930
1,Business,11.0,1.0,0.0,0.0,12.0,4.013378
2,Cost,6.0,1.0,0.0,0.0,7.0,2.341137
3,Design,10.0,9.0,0.0,0.0,19.0,6.354515
4,FunctionalValidity,17.0,15.0,1.0,0.0,33.0,11.036789
5,Organizational Environment,0.0,2.0,0.0,0.0,2.0,0.668896
6,Overdrawn Budget,1.0,0.0,0.0,0.0,1.0,0.334448
7,People,6.0,0.0,0.0,0.0,6.0,2.006689
8,Performance,25.0,8.0,0.0,0.0,33.0,11.036789
9,Personal,10.0,2.0,0.0,0.0,12.0,4.013378
